In [1]:
import os
# os.chdir('../')
# os.chdir(r'E:\DANONE\2019-11-15')
# os.getcwd()
%load_ext autoreload
%autoreload 2
import src.forecaster.diagnostic as diagnostic
# import src.forecaster.modelil_addbrandsearchindex as mod
import src.forecaster.modelil as mod
import src.forecaster.utilitaires as util
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 


raw_master = pd.read_csv('/data/raw/raw_master_il_20200210.csv')

# fc_eln = pd.read_csv('./data/di_forecast.csv')   


In [2]:
raw_master.tail()

,sku_wo_pkg,country,brand,tier,stage,stage_3f,date,offtake_di,offtake_eib,offtake_il,...,total_vol,if_vol,fo_vol,gum_vol,cl_vol,il_vol,0to6_month_population,6to12_month_population,12to36_month_population,sellin_il
2455,UK_APT_PN_4,UK,APT,PN,4,GUM,12/1/2019,23.4408,36.161759,59.602559,...,38279.12204,6352.147033,7779.149189,24147.82582,29764.25296,8514.869082,620.002194,887.467533,3336.77082,33.418519
2456,UK_C&G_C&G_1,UK,C&G,C&G,1,IF,12/1/2019,8.4016,21.185492,29.587092,...,38279.12204,6352.147033,7779.149189,24147.82582,29764.25296,8514.869082,620.002194,887.467533,3336.77082,16.372090
2457,UK_C&G_C&G_2,UK,C&G,C&G,2,FO,12/1/2019,13.6704,11.589102,25.259502,...,38279.12204,6352.147033,7779.149189,24147.82582,29764.25296,8514.869082,620.002194,887.467533,3336.77082,18.346935
2458,UK_C&G_C&G_3,UK,C&G,C&G,3,GUM,12/1/2019,37.8984,18.005758,55.904158,...,38279.12204,6352.147033,7779.149189,24147.82582,29764.25296,8514.869082,620.002194,887.467533,3336.77082,55.565425
2459,UK_C&G_C&G_4,UK,C&G,C&G,4,GUM,12/1/2019,27.4408,68.476276,95.917076,...,38279.12204,6352.147033,7779.149189,24147.82582,29764.25296,8514.869082,620.002194,887.467533,3336.77082,22.613759


In [3]:
mod = mod.Modelil(raw_master)


max_date_available = mod.all_sales.calendar_yearmonth.max()
filter_date = min(201912, max_date_available)
dwps = util.create_list_period(201601, filter_date, False)
dwp, dtp = util.get_all_combination_date(dwps, 12)

print("creating the main table") 
table_all_features = mod.create_all_features(dwp, dtp)
# table_all_features.to_csv('./data/feature_table_1019.csv')  

creating the main table


In [26]:
table_all_features[:2]

,country,brand,tier,stage,label,sku_wo_pkg,date_when_predicting,date_to_predict,country_brand_tier_stage_label_sales12,country_brand_tier_stage_label_sales11,...,sell_in_stage_sales2,sell_in_stage_sales1,month,sin_month,cos_month,target,di,eib,il,horizon
0,ANZ,APT,PF,1,di,ANZ_APT_PF_1,201601,201602,0.000000,0.000000,...,16.64,1572.242551,0.166667,0.5,0.866025,0.000000,1,0,0,1
1,ANZ,APT,PF,1,eib,ANZ_APT_PF_1,201601,201602,26.277084,17.781471,...,16.64,1572.242551,0.166667,0.5,0.866025,22.343157,0,1,0,1


In [4]:
# table_all_features = pd.read_csv('./data/feature_table.csv')  

In [5]:
from copy import deepcopy

XGBOOST_PARAMETERS = {
            'max_depth': 11,  # 25
            'gamma': 0.02,
            'subsample': 0.4,
            'n_estimators': 27,
            'learning_rate': 0.1,
            'n_jobs': 12,
            'verbosity': 2
        }
dico_params_eib = {}
for i in range(1,13):
    dico_params_eib[i] = XGBOOST_PARAMETERS.copy()

#dico_params[7]['n_estimators'] = 49
dico_params_eib[5]['n_estimators'] = 27 
dico_params_eib[6]['n_estimators'] = 45
dico_params_eib[7]['n_estimators'] = 31  
dico_params_eib[8]['n_estimators'] = 47 
dico_params_eib[9]['n_estimators'] = 13 
dico_params_eib[10]['n_estimators'] = 27

dico_params_di = deepcopy(dico_params_eib)
dico_params_di[2]['n_estimators'] = 350
dico_params_di[3]['n_estimators'] = 350
dico_params_di[4]['n_estimators'] = 120
dico_params_di[5]['n_estimators'] = 91
dico_params_di[6]['n_estimators'] = 81


In [6]:
params={}
params['eib']= dico_params_eib
params['di'] = dico_params_di

In [7]:
dwp_test = util.create_list_period(201601, 201912, False) 

In [ ]:
res = mod.recreate_past_forecasts(table_all_features, dwp_test, params, 201912,12)   

In [ ]:
# mod.feature_importance 

In [ ]:
# res.head()

In [ ]:
res.to_csv('./data/res_20200210.csv')  

In [ ]:
res = pd.read_csv('./data/res_20200210.csv')   

In [ ]:
res.head()

In [ ]:
# res['date_to_predict'] = pd.to_datetime(res['date_to_predict'])  
test = diagnostic.Diagnostic(cvr=res, raw_master=raw_master, postprocess='indep')

# res.to_csv('./data/cvr.csv')  
test.cvr.to_csv('./data/cvr_20200210.csv')  
temp = test.run_test(plot=False, prediction_horizon=6)  

In [ ]:
# month_min=24 # minimal age in month at date when predicting
# pred_horizon = 3
# test = diagnostic.Diagnostic(cvr=res[(res.horizon==3)], raw_master=raw_master, postprocess='indep')
# temp = test.run_test(plot=False, prediction_horizon=3, MINIMUM_HISTO = month_min + pred_horizon)

In [ ]:
# month_min=24 # minimal age in month at date when predicting
# pred_horizon = 6
# test = diagnostic.Diagnostic(cvr=res[(res.horizon==6)], raw_master=raw_master, postprocess='indep')
# temp = test.run_test(plot=False, prediction_horizon=6, MINIMUM_HISTO = month_min + pred_horizon)

In [ ]:
# from src.forecaster.utilitaires import fa_score, fa_score_prime, bias_score, add_sku_age
# master_il = add_sku_age(raw_master, 'date', ['sku_wo_pkg'],
#                                          ['offtake_di', 'offtake_il', 'offtake_eib'],
#                                          Thre_abs=12, Thre_rel=0.02)



# cvr = test.ensure_output_format(res[(res.horizon==6)], True)

# age = master_il[['date', 'sku_wo_pkg', 'country', 'brand', 'tier', 'stage', 'age_offtake_di',
#                               'age_offtake_il', 'age_offtake_eib']][master_il.date >= cvr.date.min()]
# age = age.rename(columns={'date': 'date'})
# age['date'] = pd.to_datetime(age['date'], format='%Y%m')

# cvr = cvr.merge(age, on=['date', 'sku_wo_pkg'], how='left', validate='m:1').fillna(-99)

In [ ]:
# cvr_ 

In [ ]:
# num = 0
# for i in [
# #     201807,201808,201809,201810,201811,201812,
# #           201901,
#           201902, 201903,201904,201905,201906,
#           201907,201908,201909,201910,201911,201912,
#           202001,202002,202003,202004,202005,202006,
#           202007,202008,202009,202010,202011,202012]:

#     cvr_ = cvr[(cvr.date_when_predicting_x==201902)&(cvr.age_offtake_di>=(30-num))]     
#     print( 'month when get actuals '+ str(i) )
#     print('mature sku number:'+ str(cvr_['sku_wo_pkg'].nunique()))
#     print(cvr_['sku_wo_pkg'].unique())  
#     print(cvr_.target_di.sum()/(cvr[(cvr.date_when_predicting_x==201902)].target_di.sum()))
#     num = num +1   

In [ ]:
# # res['date_to_predict'] = pd.to_datetime(res['date_to_predict'])  
# test = diagnostic.Diagnostic(cvr=res, raw_master=raw_master, postprocess='indep')
# x
# # res.to_csv('./data/cvr.csv')  
# test.cvr.to_csv('./data/cvr_1115.csv')  
# temp = test.run_test(plot=False, prediction_horizon=3) 

In [ ]:
# temporary_ANZ= res[res.sku_wo_pkg.isin(['ANZ_APT_PN_3', 'ANZ_APT_PN_4', 'ANZ_APT_PN_2'])]
# print(temporary_ANZ.sku_wo_pkg.unique())

# test = diagnostic.Diagnostic(cvr=temporary_ANZ, raw_master =raw_master, postprocess='indep')
# temp = test.run_test(plot=False, prediction_horizon=3)

In [ ]:
# df['date'] = pd.to_datetime(df['date'])

In [ ]:
# df.to_csv('C:/Users/zhaozi/AF_New_Model/Data/INP_CHI_FRC_IL_OFFTAKE_CHANNEL_2.csv') 